In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# evaluate multinomial logistic regression model
import numpy as np
import pandas as pd
from sklearn.model_selection import *

# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

import plotly.express as px
from IPython.display import display
import statsmodels.api as sm

<IPython.core.display.Javascript object>

In [8]:
data = pd.read_csv("Data/data_clean.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "gender", "age"], axis=1)
Y = data["gender"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,Confidence in Success,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age
0,18,2,2,2,2,1,2,1,2,2,...,1,1,2,2,2,1,2,211,1,50
1,6,2,2,2,1,2,2,1,2,1,...,2,1,2,2,2,2,1,149,1,40
2,27,1,2,2,1,2,1,2,1,2,...,1,2,1,1,2,1,2,168,1,28
3,29,1,1,2,2,2,1,2,1,1,...,1,2,1,2,2,1,1,230,1,37
4,6,1,2,1,1,1,2,1,2,1,...,2,1,2,2,2,0,1,389,1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11238,1,2,2,2,1,1,2,1,2,1,...,2,1,2,2,2,2,1,243,1,24
11239,10,2,2,1,1,1,2,1,1,1,...,2,1,2,1,2,2,1,197,1,39
11240,6,1,2,2,1,1,2,1,2,1,...,2,1,2,2,2,2,1,447,2,33
11241,12,2,2,1,1,1,1,1,1,1,...,1,2,2,2,2,2,1,167,1,24


<IPython.core.display.Javascript object>

In [13]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(multi_class="multinomial", solver="sag", fit_intercept=False)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

# print(np.argmin(n_scores["test_score"]))

Mean Accuracy: 0.637 (+/- 0.017)


<IPython.core.display.Javascript object>

In [32]:
# for model in n_scores["estimator"]:
#     print(model.coef_)

# n_scores["estimator"].coef_

model.fit(X_train, Y_train)

#Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = ["Questions", "Male", "Female", "Other"]

#Create dataframe to assess predictions 
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": model.predict(X_test)})
display(model_eval)
display(model_coef)

,Actual,Predicted
5036,2,2
4534,1,2
10026,1,2
6279,2,2
2992,1,1
...,...,...
9634,1,1
10230,2,2
1427,1,1
9871,1,1


,Questions,Male,Female,Other
0,Influence,-0.105751,-0.070539,0.176290
1,Modesty,-0.115034,0.087557,0.027477
2,Daring,-0.013032,-0.008413,0.021445
3,Confidence,0.143140,-0.069926,-0.073214
4,Ruling,0.199944,-0.298923,0.098979
5,Responsibility,0.212953,0.297788,-0.510741
6,IntrovExtro,-0.117695,0.182151,-0.064457
7,Success,0.094105,0.020682,-0.114787
8,Humble,0.217657,0.152617,-0.370274
9,Leadership,-0.075576,-0.067350,0.142926


<IPython.core.display.Javascript object>

In [6]:
# Using Statsmodels.api
logit_model = sm.MNLogit(Y, X)
result = logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.646425
         Iterations 10
                            Results: MNLogit
Model:                MNLogit            Pseudo R-squared:   0.079      
Dependent Variable:   gender             AIC:                14535.2093 
Date:                 2022-05-02 19:20   BIC:                15120.5221 
No. Observations:     11119              Log-Likelihood:     -7187.6    
Df Model:             78                 LL-Null:            -7805.3    
Df Residuals:         11039              LLR p-value:        1.2440e-207
Converged:            1.0000             Scale:              1.0000     
No. Iterations:       10.0000                                           
------------------------------------------------------------------------
       gender = 0        Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
------------------------------------------------------------------------
              Influence -0.0257   0.0469  -0.5483

<IPython.core.display.Javascript object>